# Suspicious Compression Tool Parameters
Detects suspicious command line arguments of common data compression tools

## Rule Content
```
- title: Suspicious Compression Tool Parameters
  id: 27a72a60-7e5e-47b1-9d17-909c9abafdcd
  status: experimental
  description: Detects suspicious command line arguments of common data compression
    tools
  references:
  - https://twitter.com/SBousseaden/status/1184067445612535811
  tags:
  - attack.exfiltration
  - attack.t1020
  - attack.t1002
  author: Florian Roth, Samir Bousseaden
  date: 2019/10/15
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      OriginalFileName:
      - 7z*.exe
      - '*rar.exe'
      - '*Command*Line*RAR*'
      CommandLine:
      - '* -p*'
      - '* -ta*'
      - '* -tb*'
      - '* -sdel*'
      - '* -dw*'
      - '* -hp*'
    falsepositive:
      ParentImage: C:\Program*
    condition: selection and not falsepositive
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((file_name_original.keyword:(7z*.exe OR *rar.exe OR *Command*Line*RAR*) AND process_command_line.keyword:(*\ \-p* OR *\ \-ta* OR *\ \-tb* OR *\ \-sdel* OR *\ \-dw* OR *\ \-hp*)) AND (NOT (process_parent_path.keyword:C\:\\Program*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()